In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

from keras import layers, models, optimizers, regularizers
from keras.applications import inception_v3
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# Any results you write to the current directory are saved as output.

In [ ]:
all_data=pd.read_csv('../input/train.csv')
testing=pd.read_csv('../input/test.csv')


In [ ]:
all_data=all_data.sample(frac=1).reset_index(drop=True)
all_data.head()

In [ ]:
labels=all_data['label']
labels=labels.values
all_data=all_data.drop(columns=['label'])
train_data=all_data.values

In [ ]:
train_data=train_data.astype(dtype='float32')/255
output_data=testing.values
output_data=output_data.astype(dtype='float32')/255
train_data=train_data.reshape(train_data.shape[0],28,28,1)
output_data=output_data.reshape(output_data.shape[0],28,28,1)


In [ ]:
print(train_data.shape, labels.shape)
print(type(train_data), type(labels))

In [ ]:
train_x=train_data[:36000]
train_y=labels[:36000]
train_y=to_categorical(train_y)
dev_x=train_data[36000:39000]
dev_y=labels[36000:39000]
dev_y=to_categorical(dev_y)
val_x=train_data[39000:42000]
val_y=labels[39000:42000]
val_y=to_categorical(val_y)

In [ ]:
print(train_x.shape, train_y.shape)
print(dev_x.shape, dev_y.shape)
print(val_x.shape, val_y.shape)

In [ ]:
index=1021
print(np.argmax(train_y[index]))
plt.imshow(train_x[index].reshape(28,28), cmap='gray_r')

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(28,28,1), activation='relu'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(64,(3, 3), activation='relu'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())

# Fully connected layer
model.add(layers.Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.1))
model.add(layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
from keras.preprocessing import image

In [ ]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = image.ImageDataGenerator()

In [ ]:
train_generator = gen.flow(train_x, train_y, batch_size=64)
test_generator = test_gen.flow(dev_x, dev_y, batch_size=64)

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

history = model.fit_generator(train_generator, 
                                steps_per_epoch=(36000//64), 
                                epochs=7,
            #                     batch_size=64,
                                validation_data=test_generator,
                                validation_steps=(3000//64)
                             )

In [ ]:
model.evaluate(dev_x,dev_y)

In [ ]:
prob=model.predict_proba(dev_x)

In [ ]:
index=110
print([i*100 for i in prob[index]])
print(np.argmax(prob[index]))
plt.imshow(dev_x[index].reshape(28,28))

In [ ]:
hist_dict=history.history
loss=hist_dict['loss'][2:]
val_loss=hist_dict['val_loss'][2:]
acc=hist_dict['acc']
val_acc=hist_dict['val_acc']
epochs=[i for i in range(len(loss))]
plt.plot(epochs,loss,label='Loss')
plt.plot(epochs,val_loss,label='Val acc')
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.legend()
plt.show()

model1=models.Sequential()
model1.add(layers.Conv2D(32,(3,3) ,activation='relu', input_shape=(28,28,1)))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Conv2D(64,(3,3),activation='relu'))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Conv2D(64,(3,3),activation='relu'))
model1.add(layers.Flatten())
model1.add(layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model1.add(layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model1.add(layers.Dense(10, activation='softmax'))

model1.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

history = model1.fit(train_x, 
                    train_y, 
                    epochs=20,
                    batch_size=64,
                   validation_data=(val_x,val_y))

hist_dict=history.history
loss=hist_dict['loss']
val_loss=hist_dict['val_loss']
acc=hist_dict['acc']
val_acc=hist_dict['val_acc']
epochs=[i for i in range(len(loss))]
plt.plot(epochs,acc,label='Loss')
plt.plot(epochs,val_acc,label='Val acc')
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.legend()
plt.show()

In [ ]:
preds=model.predict_classes(dev_x)

In [ ]:
wrong_x=[]
wrong_y=[]
correct_y=[]
# print(preds.shape)
# print(dev_y.shape)
for i in range(len(preds)):
#     print(preds[i], type(dev_y[i]))
    if preds[i] == np.argmax(dev_y[i]):
        pass
    else:
#         print(preds[i],np.argmax(dev_y[i]))
        wrong_x.append(dev_x[i])
        wrong_y.append(preds[i])
        correct_y.append(np.argmax(dev_y[i]))
wrong_x=np.array(wrong_x)

In [ ]:
def display_dataset(X,Y,Z=None,K=None, h=28, w=28, rows=5, cols=2, display_labels=True):
    f, ax = plt.subplots(cols, rows)
    for i in range(rows):
        for j in range(cols):
            index=np.random.randint(0,X.shape[0])
            ax[j,i].imshow(X[index].reshape(h,w), cmap='binary', )
#             tit=str(Y[index])+" "+str(int(max(K[index])*100))+", \n"+str(Z[index])
            ax[j,i].set_title(Y[index])
#     plt.axes.get_xaxis().set_visible(False)
    plt.show()


In [ ]:
display_dataset(wrong_x,wrong_y)
# print(wrong_y)

del train_x, train_y, dev_x, dev_y, val_x, val_y

In [ ]:
labels=to_categorical(labels)
labels.shape

In [ ]:
index=619
print(np.argmax(labels[index]), train_data.shape)
plt.imshow(train_data[index].reshape(28,28),cmap='gray_r')

In [ ]:
train_gen=image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

In [ ]:
train_generator = train_gen.flow(train_x, train_y, batch_size=64)

In [ ]:
clf = models.Sequential()

clf.add(layers.Conv2D(32, (3, 3), input_shape=(28,28,1), activation='relu'))
clf.add(layers.BatchNormalization(axis=-1))
clf.add(layers.Conv2D(32, (3, 3), activation='relu'))
clf.add(layers.BatchNormalization(axis=-1))
clf.add(layers.MaxPooling2D(pool_size=(2,2)))

clf.add(layers.Conv2D(64,(3, 3), activation='relu'))
clf.add(layers.BatchNormalization(axis=-1))
clf.add(layers.Conv2D(64, (3, 3), activation='relu'))
clf.add(layers.BatchNormalization(axis=-1))
clf.add(layers.MaxPooling2D(pool_size=(2,2)))

clf.add(layers.Flatten())

# Fully connected layer
clf.add(layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))
clf.add(layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
clf.add(layers.Dense(10, activation='softmax'))

clf.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

hist = clf.fit_generator(train_generator, 
                        steps_per_epoch=(42000//64), 
                        epochs=8,
    #                     batch_size=64,
                    )

print(train_data.shape,labels.shape)

In [ ]:
res=model.predict_classes(output_data)
res2=clf.predict_classes(output_data)

In [ ]:
model.fit(output_data, res2)

In [ ]:
index=4
print(res[index])
plt.imshow(output_data[index].reshape(28,28))

In [ ]:
display_dataset(output_data, res)

In [ ]:
out_lab = to_categorical(res)
out_lab.shape

In [ ]:
out_generator = test_gen.flow(output_data, out_lab, batch_size=64)

hist = clf.fit_generator(out_generator, 
                        steps_per_epoch=(28000//64), 
                        epochs=8,
    #                     batch_size=64,
                    )

In [ ]:
result=pd.DataFrame()
result['ImageId']=range(1,len(res)+1)
result['Label']=[i for i in res]
result.head()

In [ ]:
req_result = clf.predict_classes(output_data)

In [ ]:
display_dataset(output_data,req_result)

In [ ]:
result2=pd.DataFrame()
result2['ImageId']=range(1,len(req_result)+1)
result2['Label']=[i for i in req_result]
result2.head()

In [ ]:
final = result.to_csv('Submission.csv',index=False)
final = result2.to_csv('Submission_full.csv',index=False)

In [ ]:
model.to_json()